### Import all dependencies

In [1]:
import pandas as pd
import numpy as np
import cvxpy
import datetime as dt
pd.options.mode.chained_assignment = None  # default='warn'

### Define the scheduling function

In [2]:
def find_schedule(): # find_schedule(UserSched, PVsProdDetails)

    # Reading User Schedule
    user_schedule = pd.read_csv("user_schedule.csv")  # user_schedule
    #print(user_schedule.head(3))

    # Reading PhotoVoltaic Production
    pv_production = pd.read_csv("forecast.csv")
    # Removing the first column since it is not needed
    pv_production.drop(columns=pv_production.columns[0], axis=1, inplace=True)
    #print(pv_production.head(3))


    number_of_appliances = user_schedule.shape[0] # Storing number of activities that need to be allocated into time slots
    

    # Storing all the UserSchedule columns into different variables
    appliances_names = user_schedule.iloc[:,1]
    earlier_time = user_schedule.iloc[:,2]
    latest_time = user_schedule.iloc[:,3]
    task_duration = user_schedule.iloc[:,4]
    user_consumption = user_schedule.iloc[:,5]
    #appliance_importance = user_schedule.iloc[:,6]

    # Storing all the PVProd useful columns into different variables
    pv_production_date = pv_production.iloc[:,1]
    pv_production_hour = pv_production.iloc[:,5]
    pv_production_value = pv_production.iloc[:,6]
    pv_production_value = pv_production_value.values
    pv_production_value = np.expand_dims(pv_production_value,axis=0)

    number_of_discrete_moments = pv_production_value.size # Storing all the discrete production moments of the PVs

    # if the duration that the user wants is(ex. 1.5) then we have to convert it to an integer(2).
    for i in range(len(task_duration)):
        task_duration[i] = np.ceil(task_duration[i])
        
    #-------------------------------------------------------
    # ----------- SOLUTION -----------------------
    #-------------------------------------------------------

    # create a matrix that indicates the times that an appliance is allowed to operate
    allowed_operational_hours= np.zeros((number_of_appliances,number_of_discrete_moments),dtype=int)

    # supporting variable for the consumption constraints
    supporting_consumption_values= user_consumption.values.astype(float)

    # supporting variable for importance constraints

    #supporting_importance_values = appliance_importance.values.astype(float)

    # fix the dimensions
    supporting_consumption_values=np.expand_dims(supporting_consumption_values, axis=0)
    #supporting_importance_values=np.expand_dims(supporting_importance_values,axis=1)
    

    # we are now going to fill the discrete moments(hours) that the tasks are allowed to be in
    for appliance_x in range(0,number_of_appliances):
        for moment_x in range(0,number_of_discrete_moments):
            
            tmp_condition_after_earliest_time=\
            pv_production_hour.values[moment_x] >= \
            earlier_time[appliance_x]       
            
            tmp_condition_before_latest_time=\
            pv_production_hour.values[moment_x] <= \
            latest_time[appliance_x] + task_duration[appliance_x] # we want it to end when latest_start_time + duration
            
            if tmp_condition_after_earliest_time & tmp_condition_before_latest_time:
                allowed_operational_hours[appliance_x][moment_x]=1
                
    # we are now going to create the table that contains the timeframes in which the appliances will be optimally placed
    task_table =  cvxpy.Variable(allowed_operational_hours.shape, boolean=True)
    
    # it is time to start setting up the constraints of the problem
    # Firstly, we have to ensure that the devices operate during the hours that the user wants

    user_time_limit_constraint=\
    cvxpy.multiply(allowed_operational_hours,task_table) >= task_table

    # we use the "@" operator for matrix-matrix and matrix-vector multiplication

    max_consumption_contraint=\
    supporting_consumption_values @ task_table <= pv_production_value # we are thinking of deleting this when we evolve the project

    # With this constraint we ensure that the duration of the activity ends before it is unwanted

    #task_duration_rounded_up=np.ceil(task_duration.astype(int))
    #task_duration_rounded_up=np.expand_dims(task_duration_rounded_up.values,axis=1)

    task_duration_expanded = np.expand_dims(task_duration,axis=1) # we use this because we consider that the duration is given in hours(1,2,3...)
    max_duration_constraint=\
    task_table @ np.ones((number_of_discrete_moments,1),dtype=int) <= task_duration_expanded # in our version of the problem this should == since all the duration should be completely respected

    # We will also add a constraint for ensuring that our activities are continuous

    no_breaks_constraint=\
     task_table[:,2:allowed_operational_hours.shape[1]]- \
     task_table[:,1:allowed_operational_hours.shape[1]-1]+ \
     task_table[:,0:allowed_operational_hours.shape[1]-2] <= 1

    # Lastly, we are going to define a constraint that makes sures all of the jobs are programmed

    all_works_placed_constraint=\
    task_table @ np.ones((number_of_discrete_moments,1),dtype=int) >= 1

    # Now we are going to put all the constraints together

    problem_constraints=\
    [user_time_limit_constraint, max_duration_constraint, max_consumption_contraint,# : we ignore this constraint because worst case we are going to buy energy from an energy provider
    no_breaks_constraint,all_works_placed_constraint]

    # here we are define the unused production from the pv

    unused_panel_production=\
    cvxpy.sum(pv_production_value - cvxpy.sum(supporting_consumption_values @ task_table)) #supporting_importance_values 
    #cvxpy.sum((pv_production_value - ((supporting_consumption_values @ task_table))) - (supporting_importance_values @ task_table))

    # we can finally formulate the problem

    problem = cvxpy.Problem(cvxpy.Minimize(unused_panel_production), problem_constraints)

    # now we will solve the problem

    problem.solve(solver=cvxpy.GLPK_MI)

    #if there is a solution
    if ~np.isinf(problem.value):
        #get the final solution
        final_solution_1 = task_table.value
        control_variable = 1

    else:    
        final_solution_1 = 0
        control_variable = 0
    
    return final_solution_1,control_variable,appliances_names

### Calling the function and printing the results of the scheduler

In [3]:
control_variable = -1
solution,control_variable,appliances_names = find_schedule()
if control_variable == 1:
    print("This is the scheduler's solution : ")
    for i in range(appliances_names.size):
        print(appliances_names[i], solution[i], sep=' : ')
elif control_variable == 0:
    print("The algorithm could not find a final solution")
else:
    print("There was an error, please try again")
    

This is the scheduler's solution : 
Air Condition : [0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0.]
Air Purifier : [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
EV Car Charger : [0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
WiFi Booster : [0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
